In [2]:
import numpy as np
import h5py
import os

In [17]:
def map_to_files(query_regions):
    
    mapping = {}
    for REGION_INDEX in range(len(query_regions)):
      chrom  = query_regions[REGION_INDEX][0]
      posmin = query_regions[REGION_INDEX][1] - 131071
      posmax = query_regions[REGION_INDEX][2] + 131072
      mapping[REGION_INDEX] = f"/grand/TFXcan/imlab/data/enformer_training_data/larger_window/human/chr{chrom}_{posmin}_{posmax}.npy"

    return mapping


def get_files_by_size(directory, file_size=None):
    """
    Get a list of files in a directory that match the given size.

    :param directory: The directory to search in.
    :param file_size: The size to filter files by (in bytes).
    :return: tuple of (list of all files, list of files that match the given size)
    """
    # List to store the files with the given size
    matching_files = []
    all_files = []

    # Iterate over the files in the directory
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        
        # Check if it's a file and get its size
        if os.path.isfile(filepath):
            size = os.path.getsize(filepath)
            all_files.append((filepath, size))
            # If the file size matches, add it to the list
            if file_size is not None and size == file_size:
                matching_files.append(filepath)

    return all_files, matching_files

### Just a check of consistency with previous data

In [67]:
# Define a mapping from one-hot encoded arrays to nucleotide symbols
def decode_one_hot(one_hot_sequence, start_pos=None, window_size=None):
    nucleotide_map = ['A', 'C', 'G', 'T']

    if start_pos is not None and window_size is not None:
        one_hot_sequence = one_hot_sequence[start_pos:start_pos + window_size]

    decoded_sequence = ''.join(nucleotide_map[np.argmax(base)] for base in one_hot_sequence)
    return decoded_sequence


check_list = []
for REGION_INDEX in range(100):

    chrom  = query_regions[REGION_INDEX][0]
    posmin = query_regions[REGION_INDEX][1] - 131071
    posmax = query_regions[REGION_INDEX][2] + 131072
    seq = np.load(f"/grand/TFXcan/imlab/data/enformer_training_data/larger_window/human/chr{chrom}_{posmin}_{posmax}.npy")
    
    short_seq = sequences[REGION_INDEX]
    
    check_bool = decode_one_hot(short_seq) == decode_one_hot(seq, start_pos=131072, window_size=131072)
    check_list.append(check_bool)

## Build hdf5 file

In [ ]:
"test_human_393216.hdf5"
"train_human_393216.hdf5"
"test_mouse_393216.hdf5"
"train_mouse_393216.hdf5"

- Iterate over my seq files
- Remove those with the wrong size
- Link the sequence with the target, using the "query_regions"
- Add them one at a time

In [92]:
MOUSE_DIR = "/grand/TFXcan/imlab/data/enformer_training_data/larger_window/mouse"

In [59]:
query_regions, sequences, targets = {}, {}, {}

for dataset_tag in ["train", "test"]:
    
    file_path = f"/grand/TFXcan/imlab/data/enformer_training_data/basenji_data_h5/no_groups_popseq_revised_order/{dataset_tag}_pop_seq.hdf5"  # Replace with the actual file path
    
    with h5py.File(file_path, 'r') as hdf:

        query_regions[dataset_tag] = hdf['query_regions'][:]
        sequences[dataset_tag]     = hdf['sequence']
        targets[dataset_tag]       = hdf['target']


In [58]:
RIGHT_FILESIZE = 12583040

HUMAN_DIR = "/grand/TFXcan/imlab/data/enformer_training_data/larger_window/human"
file_sizes, good_files = get_files_by_size(HUMAN_DIR, RIGHT_FILESIZE)
good_files = set(good_files)

In [73]:
for dataset_tag in ['train', 'test']:

    # LENGTH = 1000

    kk = { k:file for k, file in map_to_files(query_regions[dataset_tag]).items() if file in good_files }
    hdf5_file_path = f"/grand/TFXcan/imlab/data/enformer_training_data/larger_window/{dataset_tag}_human.hdf5"
    total_sequences = len(kk)
    # total_sequences = LENGTH
    sequence_length = 393216
    nucleotides = 4
    
    with h5py.File(f"/grand/TFXcan/imlab/data/enformer_training_data/basenji_data_h5/no_groups_popseq_revised_order/{dataset_tag}_pop_seq.hdf5", "r") as old_hdf:
        with h5py.File(hdf5_file_path, 'w') as new_hdf:

            original_targets = old_hdf['target']

            sequence_dataset = new_hdf.create_dataset(
                'sequence', 
                shape=(total_sequences, sequence_length, nucleotides), 
                chunks=(1, sequence_length, nucleotides), 
                dtype='int8'
            )
    
            target_dataset = new_hdf.create_dataset(
                'target', 
                shape=(total_sequences, 896, 5313), 
                chunks=(1, 896, 5313), 
                dtype='float'
            )
        
            for i, (k, v) in enumerate(kk.items()):

                #if i == LENGTH:
                 #   break
    
                one_slice_seq    = np.load(kk[k])
                one_slice_target = original_targets[k]
                    
                # Write this slice into the dataset at position i
                sequence_dataset[i, :, :] = one_slice_seq
                target_dataset[i, :, :]   = one_slice_target
                    
                # Optionally print progress
                if i % 1000 == 0:
                    print(f"Written sequence {i} to HDF5")                
        

Written sequence 0 to HDF5
Written sequence 1000 to HDF5
Written sequence 2000 to HDF5
Written sequence 3000 to HDF5
Written sequence 4000 to HDF5
Written sequence 5000 to HDF5
Written sequence 6000 to HDF5
Written sequence 7000 to HDF5
Written sequence 8000 to HDF5
Written sequence 9000 to HDF5
Written sequence 10000 to HDF5
Written sequence 11000 to HDF5
Written sequence 12000 to HDF5
Written sequence 13000 to HDF5
Written sequence 14000 to HDF5
Written sequence 15000 to HDF5
Written sequence 16000 to HDF5
Written sequence 17000 to HDF5
Written sequence 18000 to HDF5
Written sequence 19000 to HDF5
Written sequence 20000 to HDF5
Written sequence 21000 to HDF5
Written sequence 22000 to HDF5
Written sequence 23000 to HDF5
Written sequence 24000 to HDF5
Written sequence 25000 to HDF5
Written sequence 26000 to HDF5
Written sequence 27000 to HDF5
Written sequence 28000 to HDF5
Written sequence 29000 to HDF5
Written sequence 30000 to HDF5
Written sequence 31000 to HDF5


In [ ]:

    # Create the dataset with chunks enabled (you can adjust the chunk size as needed)
    sequence_dataset = hdf.create_dataset('sequence', 
                                          shape=(total_sequences, sequence_length, nucleotides), 
                                          chunks=(1, sequence_length, nucleotides), 
                                          dtype='int8')  # Assuming one-hot is encoded as int8
    
    # Now, let's assume you have slices of data from .npy files (or another source)
    for i in range(total_sequences):
        # Load one slice of sequence data at a time
        # Here we simulate loading a slice; replace this with your actual loading mechanism
        one_slice = np.random.randint(0, 2, size=(sequence_length, nucleotides))  # Random one-hot data
        
        # Write this slice into the dataset at position i
        sequence_dataset[i, :, :] = one_slice
        
        # Optionally print progress
        if i % 100 == 0:
            print(f"Written sequence {i} to HDF5")

# After the loop, the full dataset will be saved in the HDF5 file
print("All sequences written successfully.")
